In [1]:
import sys

# # Print the PYTHONPATH before the modifications
# for i, path in enumerate(sys.path):
#     print(i, path)
# print()

# Add the new paths separately
new_path1 = "/home/michele/.local/lib/python3.10/site-packages/"
new_path2 = "/home/michele/iac_code/michele_mmdet3d/"                           # TODO: Need to add in the .bashrc
if not new_path1 in sys.path:
    sys.path.insert(1, new_path1)
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)

# # Print the PYTHONPATH after the modifications
# for i, path in enumerate(sys.path):
#     print(i, path)
# print()

# Print the version of MMCV
import mmcv
print(mmcv.__version__)
print(mmcv.__file__)

2.1.0
/home/michele/.local/lib/python3.10/site-packages/mmcv/__init__.py


In [2]:
from mmdet3d.apis import LidarDet3DInferencer

In [3]:
#################################################################################################################
#                                           Initialize Inferencer                                               #
#################################################################################################################



inferencer = LidarDet3DInferencer(model='/home/michele/iac_code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py', 
                                  weights='/home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth')

Loads checkpoint by local backend from path: /home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth
10/03 16:54:47 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.


/home/michele/iac_code/michele_mmdet3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [4]:
#################################################################################################################
#                                               Just Inference                                                  #
#################################################################################################################



# Read the files in validation list
val_list_txt_file = "/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/ImageSets/val.txt"
with open(val_list_txt_file, 'r') as file:
    val_file_names = [line.strip() for line in file]

# Choose a smaller set of the validation files
one_every_n = 1
max_number = 2
inputs = []
for i, name in enumerate(val_file_names):
    if i%one_every_n==0 and len(inputs)<max_number:
        inputs.append(dict(points=("/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/training/velodyne/"+name+".bin")))

print(f"Total validation point_clouds: {len(val_file_names)}")
print(f"\tOne every n: {one_every_n}")
print(f"\tMax number: {max_number}")
print(f"\tSelected point_clouds: {len(inputs)}")

# Initial format for the inputs was the following:
# inputs = dict(points=("/home/michele/iac_code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py"))
results = []
for input in inputs:
    results.append(inferencer(input))

Output()

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming 
release, it will be required to pass the indexing argument. (Triggered internally at 
../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Output()

In [5]:
#################################################################################################################
#                                           Inference and Visualize                                             #
#################################################################################################################



# NOTE by Michele:  The visualization does not work properly if it gets passed a list. So just cycle through the 
#       "__call__" funcion of the inferencer
for input in inputs:
    inferencer(input, show=True, wait_time = 10)

Output()

Visualizing file with timestamp:        1723235388718397661

Output()

Visualizing file with timestamp:        1723235389968373008

[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not 
match.

In [8]:
#################################################################################################################
#                                              Remote Inferencer                                                #
#################################################################################################################



import os
out_directory = '/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/remote_inference/'

# NOTE by Michele:  Same as above (need to use the for loop)
for input in inputs:
    inferencer(input, show=False, out_dir=out_directory)

# Visualize the predictions from the saved files
local_inferencer = LidarDet3DInferencer(model='/home/michele/iac_code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py',
                                        weights='/home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth')

saved_predictions = [(out_directory + "preds/" + filename) for filename in os.listdir(out_directory + "preds/")]
local_inferencer.visualize_preds_fromfile(saved_predictions, show=True, wait_time = 10)

Output()

Visualizing file with timestamp:        1723235388718397661

Output()

Visualizing file with timestamp:        1723235389968373008

/home/michele/iac_code/michele_mmdet3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


Loads checkpoint by local backend from path: /home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth
	Visualizing file with timestamp:	1723235389968373008
	Visualizing file with timestamp:	1723235388718397661
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.


# Model analysis
### Components of the model inside MMDetection3D:
- General type of net is "VoxelNet" with the following sub-components:
    1. data_preprocessor
    2. voxel_encoder
    3. middle_encoder
    4. backbone
    5. neck
    6. bbox_head